# Silver Transformation (Normalization) Silver > Gold Layer

In [21]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from datetime import datetime


StatementMeta(, e9d093ef-482b-4f5c-a580-7a65a00e8083, 23, Finished, Available, Finished)

In [22]:
# Toggle parameters

PROCESSING_START_TIME = datetime.now()


SILVER_PATH = "abfss://fabric_dev@onelake.dfs.fabric.microsoft.com/fabric_LH_sales.Lakehouse/Tables/dbo/Silver"
SILVER_PLUS_PATH = "abfss://fabric_dev@onelake.dfs.fabric.microsoft.com/fabric_LH_sales.Lakehouse/Tables/SilverPlus"
dim_customer_path = f"{SILVER_PLUS_PATH}/dim_customer"
dim_product_path = f"{SILVER_PLUS_PATH}/dim_product"
dim_state_path = f"{SILVER_PLUS_PATH}/dim_state"
dim_date_path = f"{SILVER_PLUS_PATH}/dim_date"
fact_sales_path = f"{SILVER_PLUS_PATH}/fact_sales"
mv_fact_sales_path = "abfss://fabric_dev@onelake.dfs.fabric.microsoft.com/fabric_LH_sales.Lakehouse/Tables/dbo/mv_fact_sales"


try:
    import mssparkutils
    SILVER_PATH = mssparkutils.env.getJobInput("SILVER_PATH") or SILVER_PATH
    SILVER_PLUS_PATH = mssparkutils.env.getJobInput("SILVER_PLUS_PATH") or SILVER_PLUS_PATH
    dim_customer_path = mssparkutils.env.getJobInput("dim_customer_path") or dim_customer_path
    dim_product_path = mssparkutils.env.getJobInput("dim_product_path") or dim_product_path
    dim_state_path = mssparkutils.env.getJobInput("dim_state_path") or dim_state_path
    dim_date_path = mssparkutils.env.getJobInput("dim_date_path") or dim_date_path
    fact_sales_path = mssparkutils.env.getJobInput("fact_sales_path") or fact_sales_path
    mv_fact_sales_path = mssparkutils.env.getJobInput("mv_fact_sales_path") or mv_fact_sales_path
    print(" Parameters loaded from Fabric Pipeline runtime.")
except ImportError:
    print(" Running outside Fabric — using default parameter values.")

PROCESSING_START_TIME = datetime.now()

print(f"Silver Path: {SILVER_PATH}")
print(f"Silver Plus Path: {SILVER_PLUS_PATH}")
print(f"Dim Customer Path: {dim_customer_path}")
print(f"Dim product path: {dim_product_path}")
print(f"Dim state path: {dim_state_path}")
print(f"Dim date Path: {dim_date_path}")
print(f"Fact sales Path: {fact_sales_path}")
print(f"mv fact sale Table: {mv_fact_sales_path}")

StatementMeta(, e9d093ef-482b-4f5c-a580-7a65a00e8083, 24, Finished, Available, Finished)

 Running outside Fabric — using default parameter values.
Silver Path: abfss://fabric_dev@onelake.dfs.fabric.microsoft.com/fabric_LH_sales.Lakehouse/Tables/dbo/Silver
Silver Plus Path: abfss://fabric_dev@onelake.dfs.fabric.microsoft.com/fabric_LH_sales.Lakehouse/Tables/SilverPlus
Dim Customer Path: abfss://fabric_dev@onelake.dfs.fabric.microsoft.com/fabric_LH_sales.Lakehouse/Tables/SilverPlus/dim_customer
Dim product path: abfss://fabric_dev@onelake.dfs.fabric.microsoft.com/fabric_LH_sales.Lakehouse/Tables/SilverPlus/dim_product
Dim state path: abfss://fabric_dev@onelake.dfs.fabric.microsoft.com/fabric_LH_sales.Lakehouse/Tables/SilverPlus/dim_state
Dim date Path: abfss://fabric_dev@onelake.dfs.fabric.microsoft.com/fabric_LH_sales.Lakehouse/Tables/SilverPlus/dim_date
Fact sales Path: abfss://fabric_dev@onelake.dfs.fabric.microsoft.com/fabric_LH_sales.Lakehouse/Tables/SilverPlus/fact_sales
mv fact sale Table: abfss://fabric_dev@onelake.dfs.fabric.microsoft.com/fabric_LH_sales.Lakehouse/T

In [23]:
print(" Reading from Silver...")
silver_df = spark.read.option("mergeSchema", "true").parquet(SILVER_PATH)
print(f" Loaded {silver_df.count():,} rows from Silver.")


StatementMeta(, e9d093ef-482b-4f5c-a580-7a65a00e8083, 25, Finished, Available, Finished)

 Reading from Silver...
 Loaded 9,986 rows from Silver.


#### Dim Customer

In [24]:
PROCESSING_START_TIME_cus = datetime.now()

dim_customer = (
    silver_df.select("customer_id", "customer_name", "segment",  "region", "country", "state_name", "city")
    .dropDuplicates(["customer_id"])
    .withColumn("customer_key", F.monotonically_increasing_id())
    .withColumn("ProcessedTime", F.lit(PROCESSING_START_TIME))
)

dim_customer.write.mode("overwrite").option("mergeSchema", "true").saveAsTable("SilverPlus.dim_customer")
print(f" dim_customer created → {dim_customer_path}")

# Log: Dim_Customer
PROCESSING_END_TIME_cus = datetime.now()
duration_minutes_cus = round((PROCESSING_END_TIME_cus - PROCESSING_START_TIME_cus).total_seconds() / 60, 2)

log_data = [("SilverPlus.dim_customer", PROCESSING_START_TIME_cus, PROCESSING_END_TIME_cus, duration_minutes_cus, "Silver -> SilverPlus", dim_customer_path, dim_customer.count())]

spark.createDataFrame(log_data, ["Dataset", "Start_Timestamp", "End_Timestamp", "run_duration", "Stage", "Destination", "Row_Count"]) \
    .write.mode("append").option("mergeSchema", "true").saveAsTable("dbo.pipeline_log")
print(" Log entry added for dim_customer")



StatementMeta(, e9d093ef-482b-4f5c-a580-7a65a00e8083, 26, Finished, Available, Finished)

 dim_customer created → abfss://fabric_dev@onelake.dfs.fabric.microsoft.com/fabric_LH_sales.Lakehouse/Tables/SilverPlus/dim_customer
 Log entry added for dim_customer


In [25]:
%%sql

SELECT * FROM SilverPlus.dim_customer LIMIT 5;

StatementMeta(, e9d093ef-482b-4f5c-a580-7a65a00e8083, 27, Finished, Available, Finished)

<Spark SQL result set with 5 rows and 9 fields>

#### Dim product

In [26]:
PROCESSING_START_TIME_pro = datetime.now()

dim_product = (
    silver_df.select("product_id", "product_name", "sub_category", "category")
    .dropDuplicates(["product_id"])
    .withColumn("product_key", F.monotonically_increasing_id())
    .withColumn("ProcessedTime", F.lit(PROCESSING_START_TIME))
)

dim_product.write.mode("overwrite").option("mergeSchema", "true").saveAsTable("SilverPlus.dim_product")
print(f" dim_product created → {dim_product_path}")


# Log: dim_product
PROCESSING_END_TIME_pro = datetime.now()
duration_minutes_pro = round((PROCESSING_END_TIME_pro - PROCESSING_START_TIME_pro).total_seconds() / 60, 2)

log_data = [("SilverPlus.dim_product", PROCESSING_START_TIME_pro, PROCESSING_END_TIME_pro, duration_minutes_pro, "Silver -> SilverPlus", dim_product_path, dim_product.count())]

spark.createDataFrame(log_data, ["Dataset", "Start_Timestamp", "End_Timestamp", "run_duration", "Stage", "Destination", "Row_Count"]) \
    .write.mode("append").option("mergeSchema", "true").saveAsTable("dbo.pipeline_log")
print(" Log entry added for dim_product")


StatementMeta(, e9d093ef-482b-4f5c-a580-7a65a00e8083, 28, Finished, Available, Finished)

 dim_product created → abfss://fabric_dev@onelake.dfs.fabric.microsoft.com/fabric_LH_sales.Lakehouse/Tables/SilverPlus/dim_product
 Log entry added for dim_product


In [27]:
%%sql

SELECT * FROM SilverPlus.dim_product LIMIT 5;

StatementMeta(, e9d093ef-482b-4f5c-a580-7a65a00e8083, 29, Finished, Available, Finished)

<Spark SQL result set with 5 rows and 6 fields>

#### Dim_state

In [28]:
PROCESSING_START_TIME_st = datetime.now()

dim_state = (
    silver_df.select("state_key", "state_name", "region", "country", "latitude", "longitude")
    .dropDuplicates(["state_key"])
    .withColumn("ProcessedTime", F.lit(PROCESSING_START_TIME))
)

dim_state.write.mode("overwrite").option("mergeSchema", "true").saveAsTable("SilverPlus.dim_state")
print(f" dim_state created → {dim_state_path}")

# Log: dim_state
PROCESSING_END_TIME_st = datetime.now()
duration_minutes_st = round((PROCESSING_END_TIME_st - PROCESSING_START_TIME_st).total_seconds() / 60, 2)

log_data = [("SilverPlus.dim_state", PROCESSING_START_TIME_st, PROCESSING_END_TIME_st, duration_minutes_st, "Silver -> SilverPlus", dim_state_path, dim_state.count())]

spark.createDataFrame(log_data, ["Dataset", "Start_Timestamp", "End_Timestamp", "run_duration", "Stage", "Destination", "Row_Count"]) \
    .write.mode("append").option("mergeSchema", "true").saveAsTable("dbo.pipeline_log")
print(" Log entry added for dim_state")



StatementMeta(, e9d093ef-482b-4f5c-a580-7a65a00e8083, 30, Finished, Available, Finished)

 dim_state created → abfss://fabric_dev@onelake.dfs.fabric.microsoft.com/fabric_LH_sales.Lakehouse/Tables/SilverPlus/dim_state
 Log entry added for dim_state


In [29]:
%%sql

SELECT * FROM SilverPlus.dim_state LIMIT 5;

StatementMeta(, e9d093ef-482b-4f5c-a580-7a65a00e8083, 31, Finished, Available, Finished)

<Spark SQL result set with 5 rows and 7 fields>

#### Dim_date

In [30]:
PROCESSING_START_TIME_da = datetime.now()

dim_date = (
    silver_df
    .select(F.to_date("order_date", "dd-MM-yyyy").alias("date"))
    .dropna()
    .dropDuplicates(["date"])
    .withColumn("date_key", F.monotonically_increasing_id())
    .withColumn("year", F.year("date"))
    .withColumn("month", F.month("date"))
    .withColumn("day", F.dayofmonth("date"))
    .withColumn("day_name", F.date_format("date", "EEEE"))
    .withColumn("month_name", F.date_format("date", "MMMM"))
    .withColumn("ProcessedTime", F.lit(PROCESSING_START_TIME))
)

dim_date.write.mode("overwrite").option("mergeSchema", "true").saveAsTable("SilverPlus.dim_date")
print(f" dim_date created → {dim_date_path}")


# Log: dim_date
PROCESSING_END_TIME_da = datetime.now()
duration_minutes_da = round((PROCESSING_END_TIME_da - PROCESSING_START_TIME_da).total_seconds() / 60, 2)

log_data = [("SilverPlus.dim_date", PROCESSING_START_TIME_da, PROCESSING_END_TIME_da, duration_minutes_da, "Silver -> SilverPlus", dim_date_path, dim_date.count())]

spark.createDataFrame(log_data, ["Dataset", "Start_Timestamp", "End_Timestamp", "run_duration", "Stage", "Destination", "Row_Count"]) \
    .write.mode("append").option("mergeSchema", "true").saveAsTable("dbo.pipeline_log")
print(" Log entry added for dim_date")


StatementMeta(, e9d093ef-482b-4f5c-a580-7a65a00e8083, 32, Finished, Available, Finished)

 dim_date created → abfss://fabric_dev@onelake.dfs.fabric.microsoft.com/fabric_LH_sales.Lakehouse/Tables/SilverPlus/dim_date
 Log entry added for dim_date


In [31]:
%%sql

SELECT * FROM SilverPlus.dim_date LIMIT 5;

StatementMeta(, e9d093ef-482b-4f5c-a580-7a65a00e8083, 33, Finished, Available, Finished)

<Spark SQL result set with 5 rows and 8 fields>

#### Fact_Sales

In [32]:
PROCESSING_START_TIME_fs = datetime.now()

fact_sales = (
    silver_df.select(
        "order_id",
        "customer_id",
        "product_id",
        "state_key",
        "sales",
        "profit",
        "quantity",
        "discount",
        "order_date",
        "ship_mode",
        "ship_date"
    )
    .withColumn("order_date", F.to_date("order_date", "dd-MM-yyyy"))
    .withColumn("ship_date", F.to_date("ship_date", "dd-MM-yyyy"))
    .withColumn("ProcessedTime", F.lit(PROCESSING_START_TIME))
)

fact_sales.write.mode("overwrite").option("mergeSchema", "true").saveAsTable("SilverPlus.fact_sales")

print(" fact_sales table created → silver_plus.fact_sales")


# Log: fact_sales
PROCESSING_END_TIME_fs = datetime.now()
duration_minutes_fs = round((PROCESSING_END_TIME_fs - PROCESSING_START_TIME_fs).total_seconds() / 60, 2)

log_data = [("SilverPlus.fact_sales", PROCESSING_START_TIME_fs, PROCESSING_END_TIME_fs, duration_minutes_fs, "Silver -> SilverPlus", fact_sales_path, fact_sales.count())]

spark.createDataFrame(log_data, ["Dataset", "Start_Timestamp", "End_Timestamp", "run_duration", "Stage", "Destination", "Row_Count"]) \
    .write.mode("append").option("mergeSchema", "true").saveAsTable("dbo.pipeline_log")
print(" Log entry added for fact_sales")


StatementMeta(, e9d093ef-482b-4f5c-a580-7a65a00e8083, 34, Finished, Available, Finished)

 fact_sales table created → silver_plus.fact_sales
 Log entry added for fact_sales


In [33]:
%%sql

SELECT * FROM SilverPlus.fact_sales LIMIT 5;

StatementMeta(, e9d093ef-482b-4f5c-a580-7a65a00e8083, 35, Finished, Available, Finished)

<Spark SQL result set with 5 rows and 12 fields>

In [34]:
PROCESSING_START_TIME_mv = datetime.now()

StatementMeta(, e9d093ef-482b-4f5c-a580-7a65a00e8083, 36, Finished, Available, Finished)

In [35]:
%%sql

CREATE MATERIALIZED LAKE VIEW IF NOT EXISTS dbo.mv_fact_sales AS
SELECT
    fs.order_id,
    fs.customer_id,
    c.customer_name,
    c.segment,
    fs.product_id,
    p.product_name,
    p.category,
    p.sub_category,
    fs.sales,
    fs.quantity,
    fs.profit,
    fs.discount,
    fs.order_date,
    fs.ship_mode,
    fs.ship_date,
    l.country,
    l.region,
    l.state_name AS state,
    c.city,
    l.latitude,
    l.longitude,
    d.year,
    d.month,
    d.day
FROM SilverPlus.fact_sales fs
LEFT JOIN SilverPlus.dim_customer c ON fs.customer_id = c.customer_id
LEFT JOIN SilverPlus.dim_product p ON fs.product_id = p.product_id
LEFT JOIN SilverPlus.dim_date d ON fs.order_date = d.date
LEFT JOIN SilverPlus.dim_state l ON fs.state_key = l.state_key;

StatementMeta(, e9d093ef-482b-4f5c-a580-7a65a00e8083, 37, Finished, Available, Finished)

<Spark SQL result set with 8 rows and 2 fields>

In [36]:
# Log: mv_fact_sales
from datetime import datetime

PROCESSING_END_TIME_mv = datetime.now()
duration_minutes_mv = round((PROCESSING_END_TIME_mv - PROCESSING_START_TIME_mv).total_seconds() / 60, 2)

# Try to get row count if mv_fact_sales exists
try:
    mv_fact_sales_count = spark.sql("SELECT COUNT(*) FROM dbo.mv_fact_sales").collect()[0][0]
except Exception as e:
    print(f" Could not get row count for mv_fact_sales: {e}")
    mv_fact_sales_count = 0

log_data = [(
    "dbo.mv_fact_sales",
    PROCESSING_START_TIME_mv,
    PROCESSING_END_TIME_mv,
    duration_minutes_mv,
    "SilverPlus -> Gold",
    "dbo.mv_fact_sales",
    mv_fact_sales_count
)]

log_df = spark.createDataFrame(
    log_data,
    ["Dataset", "Start_Timestamp", "End_Timestamp", "run_duration", "Stage", "Destination", "Row_Count"]
)

log_df.write.mode("append").option("mergeSchema", "true").saveAsTable("dbo.pipeline_log")

print(f" Log entry added for mv_fact_sales (Row count: {mv_fact_sales_count})")


StatementMeta(, e9d093ef-482b-4f5c-a580-7a65a00e8083, 38, Finished, Available, Finished)

 Log entry added for mv_fact_sales (Row count: 9986)


In [37]:
%%sql

SELECT * FROM SilverPlus.fact_sales LIMIT 5;

StatementMeta(, e9d093ef-482b-4f5c-a580-7a65a00e8083, 39, Finished, Available, Finished)

<Spark SQL result set with 5 rows and 12 fields>

#### Logging Pipeline

In [38]:
from datetime import datetime

PROCESSING_END_TIME = datetime.now()
duration_seconds = (PROCESSING_END_TIME - PROCESSING_START_TIME).total_seconds()
duration_minutes = round(duration_seconds / 60, 2)

log_data = [
    ("SILVER_PLUS_DATASET", PROCESSING_START_TIME, PROCESSING_END_TIME, duration_minutes,"Silver -> Silver+", SILVER_PLUS_PATH, fact_sales.count())
]
log_df = spark.createDataFrame(log_data, ["Dataset", "Start_Timestamp", "End_Timestamp", "run_duration", "Stage", "Destination", "Row_Count"])

log_df.write.mode("append").option("mergeSchema", "true").saveAsTable("dbo.pipeline_log")

print(" Pipeline load logged successfully.")


StatementMeta(, e9d093ef-482b-4f5c-a580-7a65a00e8083, 40, Finished, Available, Finished)

 Pipeline load logged successfully.
